## **示例：多智能体协作预订酒店**

在当今快节奏的世界中，规划商务旅行不仅仅是预订机票和酒店房间。它需要一种高度协调和高效的方式，而这往往很难实现。这正是多智能体系统发挥作用的地方，它彻底改变了我们管理旅行需求的方式。

想象一下，有一组智能代理随时为您服务，它们协同工作，以精准和轻松的方式处理您旅行的每个环节。借助我们先进的人工智能技术，我们创建了专门用于预订服务和行程安排的智能代理，确保您拥有无缝且无压力的旅行体验。

这是一个基本场景。在规划商务旅行时，我们需要与商务旅行代理商沟通，以获取机票信息、酒店信息等。通过AI智能体，我们可以构建用于预订服务的代理和用于行程安排的代理，让它们协作，从而提升智能化水平。


# 初始化 Azure AI Agent 服务并从 **.env** 获取配置信息

### **.env**

创建一个 .env 文件

**.env** 文件包含 Azure AI Agent 服务的连接字符串、AOAI 使用的模型，以及对应的 Google API 搜索服务 API、ENDPOINT 等信息。

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "您的 Azure AI Agent 服务模型部署名称"

[!NOTE] 您需要一个具有 100,000 速率限制（每分钟令牌数）和 600 速率限制（每分钟请求数）的模型。

  您可以在 Azure AI Foundry 的模型和端点中获取模型。

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "您的 Azure AI Agent 服务项目连接字符串"

  您可以在 AI Foundry 门户屏幕的项目概览中获取项目连接字符串。

- **SERPAPI_SEARCH_API_KEY** = "您的 SERPAPI 搜索 API 密钥"
- **SERPAPI_SEARCH_ENDPOINT** = "您的 SERPAPI 搜索端点"

要获取 Azure AI Agent 服务的模型部署名称和项目连接字符串，您需要创建 Azure AI Agent 服务。建议直接使用[此模板](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json)进行创建。（***注意：*** Azure AI Agent 服务目前仅在有限的区域设置。建议参考[此链接](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support)设置区域）

Agent 需要访问 SERPAPI。建议通过[此链接](https://serpapi.com/searches)注册。注册后，您可以获得唯一的 API 密钥和 ENDPOINT。


# 登录到 Azure

现在您需要登录到 Azure。请在 VScode 中打开终端并运行 `az login` 命令。


# 设置

要运行此笔记本，您需要安装以下库。以下是所需库及其对应的 pip 安装命令列表：

azure-identity：用于 Azure 身份验证。  
requests：用于发送 HTTP 请求。  
semantic-kernel：用于语义内核框架（假设这是一个自定义或特定的库，您可能需要从特定来源或存储库安装它）。  


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# 解释：
import asyncio：导入 asyncio 模块，该模块为 Python 提供异步编程支持。它允许你使用 async 和 await 语法编写并发代码。
from typing import Annotated：从 typing 模块导入 Annotated 类型。Annotated 用于为类型提示添加元数据，这在验证、文档或工具等方面可能非常有用。


In [ ]:
import asyncio,os
from typing import Annotated

# 说明：
通过使用 `from dotenv import load_dotenv` 和 `load_dotenv()`，你可以轻松地在 `.env` 文件中管理配置设置和敏感信息（例如 API 密钥和数据库 URL），将它们与源代码分离。这不仅提升了应用程序的安全性，还使其更易于配置。


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# 解释：

导入语句：from azure.identity.aio import DefaultAzureCredential：这行代码从 azure.identity.aio 模块中导入了 DefaultAzureCredential 类。模块名中的 aio 表明它是为异步操作设计的。

DefaultAzureCredential 的用途：DefaultAzureCredential 类是 Azure SDK for Python 的一部分。它提供了一种默认的方式来对 Azure 服务进行身份验证。它会按照特定的顺序尝试多种身份验证方法，例如环境变量、托管身份和 Azure CLI 凭据。

异步操作：aio 模块表明 DefaultAzureCredential 类支持异步操作。这意味着你可以结合 asyncio 使用它来执行非阻塞的身份验证请求。


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# 说明：
从 semantic_kernel 包中导入了各种模块和类。以下是每个导入项的详细说明：

AgentGroupChat 来自 semantic_kernel.agents：此类负责处理与 AI 代理组聊天相关的功能。AzureAIAgent 和 AzureAIAgentSettings 来自 semantic_kernel.agents.azure_ai：

- AzureAIAgent：此类用于创建和管理利用 Azure AI 服务的 AI 代理。
- AzureAIAgentSettings：此类用于配置 AzureAIAgent 的相关设置。

TerminationStrategy 来自 semantic_kernel.agents.strategies.termination.termination_strategy：

- 此类定义了在特定条件下终止 AI 代理执行的策略。

ChatMessageContent 来自 semantic_kernel.contents.chat_message_content：

- 此类用于处理聊天消息的内容。

AuthorRole 来自 semantic_kernel.contents.utils.author_role：

- 此类定义了聊天消息中作者的不同角色。

kernel_function 来自 semantic_kernel.functions.kernel_function_decorator：

- 此装饰器用于定义内核函数，这些函数可以在 semantic kernel 框架中执行。

这些导入项为创建和管理能够在群聊环境中交互的 AI 代理提供了必要的组件，这些代理可能用于完成诸如预订酒店等任务。


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# 说明：
接下来，我们从 azure.ai.projects.models 模块中导入 CodeInterpreterTool 类。

CodeInterpreterTool：这是 Azure AI SDK 的一部分，用于在 AI 项目中解释和执行代码。它提供了运行代码片段、分析代码或将代码执行集成到 AI 工作流中的功能。  
这个导入为在项目中使用 CodeInterpreterTool 设置了必要的组件，这对于涉及动态解释和执行代码的任务可能非常有用。


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# 说明：
ApprovalTerminationStrategy 类提供了一种终止 AI 代理操作的特定策略。如果其交互历史中的最后一条消息包含“saved”这个词，代理将终止运行。这在某些场景中非常有用，例如当代理的任务在收到某项内容已“保存”的确认后被认为完成时。定义交互方法。当预订计划保存后，接收到保存信号时可以停止运行。


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# 说明：

这行代码通过调用 create() 方法初始化一个 AzureAIAgentSettings 对象，使用默认或预定义的设置。这个设置对象 (ai_agent_settings) 随后可以用于配置和管理一个 AzureAIAgent 实例。


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# 说明：
通过导入 requests 库，您可以轻松在 Python 代码中发起 HTTP 请求并与网络服务进行交互。


In [ ]:
import requests

# 说明：
这是一个用于存储 API 密钥的变量，该密钥用于访问 SERP（搜索引擎结果页面）API 服务。API 密钥是一个用于验证与您账户相关的请求的唯一标识符。

'GOOGLE_SEARCH_API_KEY'：这是一个占位符字符串。您需要将 'GOOGLE_SEARCH_API_KEY' 替换为您的实际 SERP API 密钥。

目的：这行代码的目的是将 API 密钥存储在一个变量中，以便用于验证对 SERP API 服务的请求。访问该服务并执行搜索需要 API 密钥。

如何获取 SERP API 密钥：要获取 SERP API 密钥，请按照以下一般步骤操作：https://serpapi.com（具体步骤可能因您使用的 SERP API 服务而异）：

选择一个 SERP API 服务：有多种 SERP API 服务可供选择，例如 SerpAPI、Google Custom Search JSON API 等。选择最适合您需求的服务。

注册账户：

访问所选 SERP API 服务的网站 https://www.serpapi.com 并注册一个账户。您可能需要提供一些基本信息并验证您的电子邮件地址。

创建 API 密钥：

注册后，登录您的账户并导航到 API 部分或仪表板。寻找创建或生成新 API 密钥的选项。
复制 API 密钥：

一旦生成了 API 密钥，请将其复制下来。此密钥将用于验证您对 SERP API 服务的请求。
替换占位符：

将 .env 文件中的占位符替换掉。


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# 说明：
BookingPlugin 类提供了使用 Serpapi.com 的 Google 搜索 API 预订酒店和航班的方法。它构建必要的参数，发送 API 请求，并处理响应以返回相关的预订信息。API 密钥 (SERPAPI_SEARCH_API_KEY) 和端点 (SERPAPI_SEARCH_ENDPOINT) 用于验证身份并向 Google 搜索 API 发送请求。


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# 说明：
SavePlugin 类提供了一个名为 saving_plan 的方法，用于使用 Azure AI 服务保存旅行计划。它设置了 Azure 凭据，创建了一个 AI 代理，处理用户输入以生成并保存旅行计划内容，并负责文件保存和清理操作。方法成功完成后会返回 "Saved"。


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# 说明：
此代码设置了 Azure AI 代理，用于处理预订航班和酒店，并根据用户输入保存旅行计划。它使用 Azure 凭据来创建和配置代理，通过群聊处理用户输入，并确保在任务完成后进行适当的清理。代理使用特定的插件（BookingPlugin 和 SavePlugin）来执行各自的任务。


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)



---

**免责声明**：  
本文档使用AI翻译服务[Co-op Translator](https://github.com/Azure/co-op-translator)进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。应以原始语言的文档作为权威来源。对于关键信息，建议使用专业人工翻译。我们不对因使用此翻译而产生的任何误解或误读承担责任。
